<a href="https://colab.research.google.com/github/parwinderau/DataspaceConnector/blob/main/JSONTokenizerBertModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers
!pip install tensorflow
!pip install scikit-learn


In [5]:
import json
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split

class JsonHarmonizer:
    def __init__(self, model_name='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = self.build_model()

    def build_model(self):
        input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='input_ids')
        attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='attention_mask')

        bert_model = TFBertModel.from_pretrained('bert-base-uncased')
        bert_output = bert_model(input_ids, attention_mask=attention_mask)[0]
        cls_token = bert_output[:, 0, :]  # Use the output for the [CLS] token

        dense = tf.keras.layers.Dense(128, activation='relu')(cls_token)
        output = tf.keras.layers.Dense(2, activation='softmax')(dense)  # Assuming binary classification for simplicity

        model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        return model

    def preprocess(self, json_data):
        def flatten_json(y):
            out = {}

            def flatten(x, name=''):
                if type(x) is dict:
                    for a in x:
                        flatten(x[a], name + a + '_')
                elif type(x) is list:
                    i = 0
                    for a in x:
                        flatten(a, name + str(i) + '_')
                        i += 1
                else:
                    out[name[:-1]] = x
            flatten(y)
            return out

        flat_json = [flatten_json(item) for item in json_data]
        text_data = [' '.join(f'{k} {v}' for k, v in item.items()) for item in flat_json]
        return text_data

    def tokenize(self, text_data):
        encodings = self.tokenizer(text_data, truncation=True, padding=True, return_tensors='tf')
        return encodings

    def train(self, json_data, labels):
        text_data = self.preprocess(json_data)
        encodings = self.tokenize(text_data)

        # Ensure all encodings and labels are tensors
        input_ids = tf.convert_to_tensor(encodings['input_ids'])
        attention_mask = tf.convert_to_tensor(encodings['attention_mask'])
        labels = np.array(labels)

        # Split the data
        X_train_ids, X_val_ids, X_train_mask, X_val_mask, y_train, y_val = train_test_split(
            input_ids.numpy(), attention_mask.numpy(), labels, test_size=0.2, random_state=42
        )

        # Convert arrays back to tensors
        X_train_ids = tf.convert_to_tensor(X_train_ids)
        X_train_mask = tf.convert_to_tensor(X_train_mask)
        X_val_ids = tf.convert_to_tensor(X_val_ids)
        X_val_mask = tf.convert_to_tensor(X_val_mask)
        y_train = tf.convert_to_tensor(y_train)
        y_val = tf.convert_to_tensor(y_val)

        # Debug: Print shapes of tensors
        print(f'X_train_ids shape: {X_train_ids.shape}')
        print(f'X_train_mask shape: {X_train_mask.shape}')
        print(f'X_val_ids shape: {X_val_ids.shape}')
        print(f'X_val_mask shape: {X_val_mask.shape}')
        print(f'y_train shape: {y_train.shape}')
        print(f'y_val shape: {y_val.shape}')

        # Training the model
        self.model.fit(
            [X_train_ids, X_train_mask], y_train,
            validation_data=([X_val_ids, X_val_mask], y_val),
            epochs=3,
            batch_size=16
        )

    def harmonize(self, json_input):
        def flatten_json(y):
            out = {}

            def flatten(x, name=''):
                if type(x) is dict:
                    for a in x:
                        flatten(x[a], name + a + '_')
                elif type(x) is list:
                    i = 0
                    for a in x:
                        flatten(a, name + str(i) + '_')
                        i += 1
                else:
                    out[name[:-1]] = x
            flatten(y)
            return out

        flat_input = flatten_json(json_input)
        text_input = ' '.join(f'{k} {v}' for k, v in flat_input.items())
        encodings = self.tokenize([text_input])
        input_ids = tf.convert_to_tensor(encodings['input_ids'])
        attention_mask = tf.convert_to_tensor(encodings['attention_mask'])

        predictions = self.model.predict([input_ids, attention_mask])
        predicted_label = np.argmax(predictions, axis=-1)

        # Placeholder harmonization logic based on predicted label
        harmonized_output = {
            "entity": flat_input.get("factory_name", flat_input.get("manufacturer_employee", flat_input.get("maker_person", ""))),
            "product": flat_input.get("factory_product", flat_input.get("manufacturer_item", flat_input.get("maker_goods", "")))
        }
        return harmonized_output

# Example usage
json_data = [
    {'factory': {'name': 'Alice', 'product': 'car'}},
    {'manufacturer': {'employee': 'Alice', 'item': 'car'}}
]

# Labels for training, assuming a simple binary classification task
labels = [0, 1]

harmonizer = JsonHarmonizer()
harmonizer.train(json_data, labels)
input_json = {'maker': {'person': 'Charlie', 'goods': 'computer'}}
harmonized_output = harmonizer.harmonize(input_json)
print(harmonized_output)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

X_train_ids shape: (1, 10)
X_train_mask shape: (1, 10)
X_val_ids shape: (1, 10)
X_val_mask shape: (1, 10)
y_train shape: (1,)
y_val shape: (1,)
Epoch 1/3


1/1 [==============================] - 43s 43s/step - loss: 0.5551 - accuracy: 1.0000 - val_loss: 0.9213 - val_accuracy: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 2s 2s/step - loss: 0.3353 - accuracy: 1.0000 - val_loss: 1.5027 - val_accuracy: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 4s 4s/step
{'entity': 'Charlie', 'product': 'computer'}
